In [19]:
import pandas as pd
import re 

import requests
from bs4 import BeautifulSoup
import time

In [29]:
title = []
category = []
subcategory = []
# city = [] город не нужен, каталог не зависит от выбранного города т.к сайт статический а выбор города сохраняется в local storage
price = []
about_price = []
offer = []
old_price = []

In [30]:
start = time.time()
page = requests.get('https://kingfisher.kz/')
soup = BeautifulSoup(page.text, 'html.parser')

listers = soup.find_all(class_ = 'dropmenu')

for category_l in listers:
    listers_sub = category_l.find_all('a')
    for subcategory_l in listers_sub:
        page2 = requests.get('https://kingfisher.kz' + subcategory_l['href'])
        soup2 = BeautifulSoup(page2.text, 'html.parser')
        
        items = soup2.find_all(class_ = 'goodsBlock')
        
        for item in items:
            title.append(item.find(class_ = 'title').text.strip())
            category.append(category_l.find('span').text.strip())
            subcategory.append(subcategory_l.text.strip())
            about_price.append(item.find(class_ = 'pricePosition').find(class_='text').text.strip())
            if re.search("Акция", item.text) != None:
                offer.append(1)
                temp = item.find(class_ = 'price').text.strip().split('\n')
                price.append(int(temp[0].replace(' ','').replace('Т', '')))
                old_price.append(int(temp[1].replace(' ','').replace('Т', '')))
            else:
                offer.append(0)
                price.append(int(item.find(class_ = 'price').text.strip().replace(' ','').replace('Т', '')))
                old_price.append(None)

print('Скрап выполнен за:',time.time() - start)

Скрап выполнен за: 32.6651873588562


In [31]:
df = pd.DataFrame()
df['title'] = title
df['category'] = category
df['subcategory'] = subcategory
df['price'] = price
df['about_price'] = about_price
df['old_price'] = old_price
df['offer'] = offer
df

,title,category,subcategory,price,about_price,old_price,offer
0,Креветки Королевские 26/30 очищенные,Морепродукты,Креветки,5290,Цена за 1 шт.,6100.0,1
1,Креветка Королевская 31/40 очищенная,Морепродукты,Креветки,5900,Цена за 1 шт.,NaN,0
2,"Лангустины в панцире с головой, размер 21-30",Морепродукты,Креветки,6500,Цена за 1 кг.,NaN,0
3,"Креветка Королевская 21/25, в панцире без головы",Морепродукты,Креветки,5500,Цена за 1 шт.,NaN,0
4,Чука салат замороженный,Морепродукты,Креветки,2800,Цена за 1 шт.,NaN,0
...,...,...,...,...,...,...,...
455,Масло Авокадо Avocado Oil,Масло,Авокадо Масло,4000,Цена за 1 шт.,NaN,0
456,Масло Сливочное BURRO ASOLO,Масло,Масло Сливочное,1790,Цена за 1 шт.,1890.0,1
457,Сковорода для Паэльи 38 см,Кухня,Сковороды VAELLA,9000,Цена за 1 шт.,NaN,0
458,Чугунная Сковорода Гриль VAELLO,Кухня,Сковороды VAELLA,12500,Цена за 1 шт.,NaN,0


In [32]:
df.to_csv('results.csv')